In [ ]:
# load libraries and scripts
import JSON
using Plots
using HDF5
using Trapz
using Statistics
using CairoMakie
using SparseArrays
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/stable.jl")
include("../scripts/dynamics.jl")

In [ ]:
#load borders
border = import_border("../data/borders/border.json")
border = border
# create the lattice grid
dx = 20 
Nx, Ny, xrange, yrange, isinside, isborder, n = get_grid(border, Float64(dx))
isgrid = isinside .| isborder
# define x and y coordinate for plotting
x = repeat(reshape(xrange,1,Nx), Ny, 1)
y = repeat(reshape(yrange,Ny,1), 1, Nx);


In [ ]:
# compute or load grid parameters
recompute_param = true
if(recompute_param || isfile("../numerics/grid_params_" * string(dx) * ".h5"))
    bx, by, p, m, d = get_params_diff(isinside, n, Float64(dx), yrange, xrange, "../data/disc/pantagruel.h5",
        "../numerics/grid_params_" * string(dx) * ".h5", Niter = 1000, dmax = 10.0, patch=0.005, bmin=0.01)
    #bx, by, p, m, d = get_params(isinside, isborder, n, Float64(dx), yrange, xrange, "../data/pantagruel.h5",
    #    "../numerics/grid_params_" * string(dx) * ".h5", bmin=0.1, sigma=0.1)
else
    bx, by, p, m, d = get_params(isinside, "../numerics/grid_params_" * string(dx) * ".h5")
end

In [ ]:

Plots.contour(m,fill=true)
# Plots.contour(d,fill=true)
#minimum(bx[bx.>0])
#println(sum(p[isborder]))

In [ ]:
# compute, iteratively, the stable solution
th, ths = compute_stable_sol(isinside, n, bx, by, p, Niter = 20000, interval = 1000);

In [ ]:
hm_plot(isinside, xrange, yrange, th)

In [ ]:
# save the result
fid = h5open("../numerics/stable_" * string(dx) * ".h5", "w")
write(fid, "th", th)
close(fid)

In [ ]:
gen, dem, bline, idb, idgen, coord, mg, dg, dl, th_ref = load_discrete_model("../data/disc/pantagruel.h5");

In [ ]:
th2 = set_ref_phase(isinside, yrange, xrange, th, [750.0; -1000.0], th_ref = 0.0)
th_cont = get_discrete_values(isinside, yrange, xrange, th, coord);

In [ ]:
Plots.plot(th_ref, label="th_disc")
Plots.plot!(th_cont, label="th_cont")
savefig("theta.pdf")

In [ ]:
fx, fy = power_flow(isinside, th, bx, by, Float64(dx))
hm_plot(isgrid, xrange, yrange, th)

In [ ]:
# plot the 
f = Figure(resolution = (800, 800))
Axis(f[1, 1])
Makie.arrows!(x[isgrid], y[isgrid], -fx[isgrid], -fy[isgrid], linewidth = 0.5, arrowsize = 1.8, lengthscale = 100)
save("flows.pdf",f)